# Karatsuba Multiplication in Python

The Karatsuba multiplication algorithm is named after the Russian mathematician Anatoly Karatsuba. It uses a divide and conquer approach that gives it a running time improvement over the standard “grade-school” method. Here we discuss Python implementations of both algorithms and compare their running times.

## 1. The Problem

You are given two integers x and y. Both have a length of length n digits. You want to find the product of these two numbers. So you want to find z in: __z = x * y__

The size of the problem is n. The more digits in x and y the harder the problem.

Above I have assumed that both x and y have the same digit length. The problem can be extended to cases where they are not the same number of digits. To do this just add the appropriate number of zeros to the left of the smaller of the two integers.

## 2. The Grade School Algorithm

The algorithm is:

1. Break the second number into units, tens, hundreds, thousands etc.
2. Start with the units. Multiply the units from the second number by each digit in the first number.
3. Do the same with the tens but add a zero before your answer.
4. Do the same with the hundreds but add two zeros before your answer.
5. Continue through the digits of the second number adding an additional zero before your answer each time.
6. Sum all these partial multiplications to get the final answer.

## 3. Grade School Algorithm in Python

This iterative process can be coded in Python with a double for loop. The first for loop works through the digits in the second number. The second (nested) for loop works through the digits in the first number.

In [8]:
# Return string with zeros added to the left or right
def zeroPad(numberString, zeros, left = True):
    for i in range(zeros):
        if left:
            numberString + "0" + numberString
        else:
            numberString = numberString + "0"
    return numberString

# Multiply two integers using the grade school algorithm
def gradeSchoolMultiplication(x, y):
    # Convert to strings for easy access to digits
    x = str(x)
    y = str(y)
    
    # Track number of zeros required to pad during partial multiplications
    zeroPadding = 0
    
    # Sum partial multiplications
    partialSum = 0
    
    # Track sum of partial multiplications
    for i in range(len(y) - 1, -1, -1):
        
        # Track carry number for multiplications that result in answers > 9
        carry = 0
        
        # Partial multiplication answer as a string for easier manipulation
        partial = ""
        
        # Pad with zeros on the RIGHT
        partial = zeroPad(partial, zeroPadding, False)
        
        # Loop over each digit in the first number
        for j in range(len(x) - 1, -1, -1):
            z = int(y[i]) * int(x[j])
            z += carry
            
            # Convert to string for easier manipulation
            z = str(z)
            
            # Track carry when answer > 9
            if len(z) > 1:
                carry = int(z[0])
            else:
                carry = 0
                
            # Concatenate final answer to the left of partial string
            partial = z[len(z) - 1] + partial
            
        # Concatenate any leftover carry to end of partial string
        if carry > 0:
            partial = str(carry) + partial
            
        # Sum partials
        partialSum += int(partial)
        
        # Add another zero to the right for the next digit of the second number y
        zeroPadding += 1
        
    return partialSum

In [10]:
print(gradeSchoolMultiplication(2925, 6872))

20100600


## 4. Grade School Algorithm Running Time

When thinking about running times we need to be clear what we actually mean by running time. All the calculations in the algorithm can broken down into single digit additions, subtractions and multiplications. This is what we will base our running time analysis on.

The running time is the total single digit addition, subtraction and multiplication calculations used to compute an answer. We will call an algorithm faster if it uses fewer of these basic operations to compute an answer.

How many of these basic calculations does the grade-school algorithm need? Each partial multiplication requires $n$ single digit multiplications. One for each digit in the first number. It also needs at most another n single digit additions when numbers are carried. So each partial multiplication uses at most $2n$ of the basic operations.

There are $n$ partial multiplications. One for each digit in the second number. So the maximum number of basic operations needed to calculate all partial multiplications is $2n^2$.

How about adding all the partial multiplications? How many basic operations will that use? At most $2n^2$ are needed.

In total the maximum number of basic operations the grade-school algorithm needs to use is $4n^2$. This is the worst case running time. If we get lucky and multiply two numbers that don’t need lots of “carrying” then fewer calculations will be needed.

## 5. Karatsuba's Algorithm

The key to understanding Karatsuba’s multiplication algorithm is remembering that you can express x (an n-digit integer) in the following way:

$x = a * 10^{n/2} + b$

You can use this if you want to multiply x by another n-digit integer y:

$y = c * 10^{n/2} + d$

Then x multiplied by y can be written as:

$xy = (a * 10^{n/2} + b)(c * 10^{n/2} + d)$

$xy = ac * 10^n + (ad + bc) * 10^{n/2} + bd$

This is where Karatsuba found a neat trick. He found a way to calculate ac, bd and (ad + bc) with just three multiplications (instead of four).

To see why think about:

$(a + b)(c + d) = ac + ad + bc + bd$

If you have already calculated ac, and bd then $(ab + bc)$ can be calculated by subtracting $ac$ and $bd$ from $(a+b)(c+d)$:

$(a + b)(c + d) - ac - bd = ad + bc$

You can use Karatsuba’s trick recursively to compute the multiplication - below are the algorithm steps:

1. Break the two integers $x$ and $y$ into $a$, $b$, $c$ and $d$ as described above
2. Recursively compute $ac$
3. Recursively compute $bd$
4. Recursively compute $(a + b)(c + d)$
5. Calculate $(ab + bc)$ as $(a + b)(c + d) – ac – bd$
6. Let A be $ac$ with $n$ zeros added to the end
7. Let B be $(ab + bc)$ with half $n$ zeros added to the end
8. The final answer is $A + B + bd$

By “recursively compute” I mean call the algorithm again to calculate these multiplications. For recursion you always need a base case to prevent an endless loop. The base case here is when the two integers x and y are single digit. In this case the algorithm just calculates and and returns their product.

__Warning__: this is only valid for integers with an even number of digits. You can easily extend it to be correct for odd digit integers. You just need to make sure there is consistency between the number of digits in b and d and the zeros you add to get A and B.

## 6. Karatsuba Example

Let’s apply the Karatsuba algorithm to calculate the product of $2925$ and $6872$.

First break the 2925 into 2 chunks, $a = 29$ and $b = 25$. Then break the second integer into two chunks, $c = 68$ and $d = 72$.

Then calculate $ac$ as $29 * 68 = 1972$, $bd$ as $25 * 72 = 1800$ and $(a + b) * (c + d)$ as $(29 + 25) * (68 + 72) = 54 * 140 = 7560$.

Next subtract $ac$ and $bd$ from the final quantity to get $(ad + bc) = 7560 – 1972 – 1800 = 3788$.

Add 4 zeros to the end of $ac$ to get $19,720,000$ call this number $A$.

Add 2 zeros to the end of $(ab + cd)$ to get $378,800$ call this number $B$.

Then sum $A = 19,720,000$, $B = 378,800$ and $bd = 1800$ to get $20,100,600$.

This result matches exactly what we got with the grade-school algorithm.

## 7. Karatsuba Multiplication Algorithm in Python

Below is an implementation of Karatsuba's algorithm. The zeroPad() function defined above for the grade-school algorithm is used. $k$ is the variable used to hold the value of $(a + b)(c + d)$.

Note use of recursion when calculating $ac$, $bd$ and $k$. This works because a base case is specified for single digit integers.

In [72]:
# Multiply two integers using Karatsuba's multiplication algorithm
def karatsubaMultiplication(x, y):
    
    # Convert to strings for easy access to digits
    x = str(x)
    y = str(y)
    
    # Base case for recursion
    if len(x) == 1 and len(y) == 1:
        return int(x) * int(y)
    if len(x) < len(y):
        x = zeroPad(x, len(y) - len(x))
    elif len(y) < len(x):
        y = zeroPad(y, len(x) - len(y))

    n = len(x)
    j = n//2
    
    # For odd digit integers
    if (n % 2) != 0:
        j += 1
        
    BZeroPadding = n - j
    AZeroPadding = BZeroPadding * 2
    
    a = int(x[:j])
    b = int(x[j:])
    c = int(y[:j])
    d = int(y[j:])
    
    print(a, b, c, d)
    
    # Recursively calculate
    ac = karatsubaMultiplication(a, c)
    bd = karatsubaMultiplication(b, d)
    k = karatsubaMultiplication(a + b, c + d)
    A = int(zeroPad(str(ac), AZeroPadding, False))
    B = int(zeroPad(str(k - ac - bd), BZeroPadding, False))
    
    return A + B + bd
    print(A + B + bd)

In [75]:
# Multiply two integers using Karatsuba's multiplication algorithm
# https://stackoverflow.com/questions/42324419/karatsuba-multiplication-implementation
def karatsuba(x, y):
    
    # Base case for recursion
    if len(str(x)) == 1 or len(str(y)) == 1:
        # print (x * y)
        return x * y
    
    else:
        m = max(len(str(x)),len(str(y)))
        m2 = m // 2
        
        # Use // to take first digit group and % to take second digit group in both x and y
        a = x // 10**(m2)
        b = x % 10**(m2)
        c = y // 10**(m2)
        d = y % 10**(m2)
        
        # print (a, b, c, d)

        # Recursively calculate
        A = karatsuba(a, c)
        B = karatsuba(a + b, c + d)
        bd = karatsuba(b, d)
        
        # print ((A * 10**(2 * m2)) + ((B - A - bd) * 10**(m2)) + bd)
        return (A * 10**(2 * m2)) + ((B - A - bd) * 10**(m2)) + bd

In [76]:
print(karatsuba(3141592653589793238462643383279502884197169399375105820974944592, 2718281828459045235360287471352662497757247093699959574966967627))

8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184


## 8. Karatsuba Running Time

To find an upper bound on the running time of Karatsuba’s algorithm we can use the master theorem. The master theorem states that the running time of a recursive algorithm is:

If $T(n) = aT(n/b) + O(n^d)$

Then:

$T(n) = O(n^d * log(n))$ if $a = b^d$

$T(n) = O(n^a)$ if $a < b^d$

$T(n) = O(n^{logb(a)})$ if $a > b^d$

Where:

-  $T(n)$ the running time of the algorithm
-  $a$ is the number of recursive calls used
-  $b$ is the scale that the problem size shrinks by with each recursive call
-  $O(n^d)$ represents the work done by the algorithm outside of the recursive calls

For Karatsuba’s algorithm we can see there are three recursive calls. On lines 25, 26 and 27. So a = 3.

Each time a recursive call is made it is on a problem of half the size. This is because of the way each number is broken into two even sized chunks. So b = 2.

Outside of the recursive calls Karatsuba’s algorithm uses a constant number of addition and subtractions. Six in total. On lines 27, 29 and 30. Each of which will use at most 3n single digit additions. So we can say the work done outside recursive calls is O(n). So d = 1.

Because a = 3 and db = 2 we know a < bd. This means the last case of the master theorem is used. So the running time of Karatsuba’s recursive algorithm is:

$T(n) = O(n^{log2(3)})$

$T(n) = O(n^{1.585})$

The Grade School algorithm had a worst case running time of $4n^2$.

$4n^2$ is $O(n^2)$. Karatsuba’s algorithm is faster with $O(n^{1.585})$ running time.

Karatsuba wins!